Importing the dependencies

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#Printing the stopword
print(stopwords.words('english'))
print(len(stopwords.words('english')))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [14]:
#loading data

twitter_data = pd.read_csv('/content/drive/MyDrive/csv/twitter_new.csv', encoding= 'ISO-8859-1')

In [9]:
#Shape of the column

twitter_data.shape

(1599999, 6)

In [10]:
# First 5 rows in dataset

twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
# Namining the columns

column_names = ['target', 'ids', 'date', 'flag', 'user', 'tweet']


In [15]:
#loading data with column names

twitter_data = pd.read_csv('/content/drive/MyDrive/csv/twitter_new.csv', names = column_names, encoding= 'ISO-8859-1')

In [16]:
twitter_data.shape

(1600000, 6)

In [17]:
twitter_data.head()

,target,ids,date,flag,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [19]:
# Counting Null values

twitter_data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
tweet     0
dtype: int64

In [20]:
# Checking the distribution of target columns

twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [23]:
# Convert target '4' into '1'

twitter_data['target'].replace(4,1, inplace = True)

In [24]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64



*   1 -> Positive Tweet
*   0 -> Negative Tweet



Stemming

In [26]:
port_stem = PorterStemmer()

In [27]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [28]:
twitter_data['stemmed_content'] = twitter_data['tweet'].apply(stemming)

In [29]:
twitter_data['stemmed_content']

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object

In [30]:
# Separating the data and label

X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [31]:
print(X)
print(Y)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


Splitting the training & testing dataset

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify= Y, random_state=5)

In [33]:
print(X_train.shape, X_test.shape)

(1280000,) (320000,)


In [34]:
# Converting textual data into numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [35]:
print(X_train, X_test)

  (0, 93843)	0.3520625811633061
  (0, 153268)	0.4594366485420904
  (0, 124737)	0.5231197021850295
  (0, 169423)	0.4307460563004172
  (0, 274638)	0.45362265936406476
  (1, 429569)	0.3606951529447222
  (1, 33752)	0.3606951529447222
  (1, 438493)	0.17920320711015286
  (1, 249196)	0.11625131115897648
  (1, 272655)	0.1922328392689911
  (1, 426531)	0.1476075287094744
  (1, 66382)	0.2524858074543787
  (1, 39493)	0.1402323392193017
  (1, 331003)	0.5650798659991657
  (1, 181713)	0.37735485447514094
  (1, 283017)	0.30050337168173064
  (2, 346877)	0.4461921387303361
  (2, 400406)	0.21359272849862912
  (2, 310001)	0.6920301869860533
  (2, 377901)	0.5257232560652112
  (3, 397282)	0.34890525194721284
  (3, 102415)	0.420743879737433
  (3, 78450)	0.3348573915248513
  (3, 127112)	0.3120936733238297
  (3, 2485)	0.3822125370657282
  :	:
  (1279996, 438991)	0.31219774225859287
  (1279996, 445613)	0.21229083138944874
  (1279997, 344980)	0.36528700223303945
  (1279997, 63204)	0.49356198783242905
  (1279997,

Training the ML Model - Logistic Regression

In [38]:
model = LogisticRegression(max_iter=1000)

In [40]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evalution

In [42]:
# Accuracy Score on training data

X_train_prediction = model.predict(X_train)
accuracy_training = accuracy_score(X_train_prediction, Y_train)

print('Accuracy score of training data : ', accuracy_training)

Accuracy score of training data :  0.810353125


In [43]:
# Accuracy Score on test data

X_test_prediction = model.predict(X_test)
accuracy_test = accuracy_score(X_test_prediction, Y_test)

print('Accuracy score of test data : ', accuracy_test)

Accuracy score of test data :  0.777621875


**Model Accuracy -> 77.76%**

Saving the model

In [44]:
import pickle

pickle.dump(model, open('twitter_sentiment_analysis.pkl', 'wb'))